In [15]:
import pytest
import torch
import torch.nn as nn
from model import LayerNorm, MLP, GPT, GPTConfig, CausalSelfAttention, Block
from copy import deepcopy
import inspect

class SplitAttention(nn.Module):
    """
    SplitAttention is a wrapper around CausalSelfAttention that splits the key, query, and value projections.
    This allows us to apply separate weight decay to the key and query projections.
    """
    def __init__(self, csa : CausalSelfAttention):
        super().__init__()

        # Check if csa.c_attn has a bias
        has_bias = csa.c_attn.bias is not None

        # key, query, value projections split into self.w_k, self.w_q, and self.w_v
        self.w_q = nn.Linear(csa.n_embd, csa.n_embd, bias=has_bias)
        self.w_k = nn.Linear(csa.n_embd, csa.n_embd, bias=has_bias)
        self.w_v = nn.Linear(csa.n_embd, csa.n_embd, bias=has_bias)

        # Copying weights from the original c_attn layer to self.w_q, self.w_k, and self.w_v
        self.w_q.weight.data.copy_(csa.c_attn.weight.data[:csa.n_embd, :])
        self.w_k.weight.data.copy_(csa.c_attn.weight.data[csa.n_embd:2*csa.n_embd, :])
        self.w_v.weight.data.copy_(csa.c_attn.weight.data[2*csa.n_embd:, :])
        
        # Copying bias from the original c_attn layer if it exists
        if has_bias:
            self.w_q.bias.data.copy_(csa.c_attn.bias.data[:csa.n_embd])
            self.w_k.bias.data.copy_(csa.c_attn.bias.data[csa.n_embd:2*csa.n_embd])
            self.w_v.bias.data.copy_(csa.c_attn.bias.data[2*csa.n_embd:])

        # The rest is the same as CausalSelfAttention, just copying
        
        # output projection
        self.c_proj = deepcopy(csa.c_proj)
        # regularization
        self.attn_dropout = deepcopy(csa.attn_dropout)
        self.resid_dropout = deepcopy(csa.resid_dropout)
        self.n_head = csa.n_head
        self.n_embd = csa.n_embd
        self.dropout = csa.dropout
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))
        
    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        k = self.w_k(x)
        q = self.w_q(x)
        v = self.w_v(x)
        
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            # manual implementation of attention
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

    # TODO: def get_magnitudes():
        """
        Calculate and return the magnitudes of all heads.
        This is equivalent to the beta (inverse temperature) in a hopfield layer.
        The magnitude of single head attention is the norm of W_k^T W_q.
        For multihead attention...
        """

class SplitGPTWrapper():
    """
    SplitGPTWrapper is a wrapper around GPT that replaces all CausalSelfAttention layers with SplitAttention.
    """
    def __init__(self, gpt_instance : GPT):
        self.gpt = gpt_instance

        # Replace all CausalSelfAttention layers with SplitAttention
        for block in self.gpt.transformer.h:
            block.attn = SplitAttention(block.attn)

    def __getattr__(self, name):
        # Delegate calls to non-overridden methods to the GPT instance
        return getattr(self.gpt, name)
    
    def __call__(self, *args, **kwargs):
        # Delegate the call to the GPT instance's forward method
        return self.gpt(*args, **kwargs)

    def configure_optimizers(self, weight_decay, qk_weight_decay, learning_rate, betas, device_type):
        # Collect parameters, applying separate weight decay for w_k and w_q in SplitAttention
        decay_params = []
        nodecay_params = []
        qk_decay_params = []
        for name, param in self.named_parameters():
            if 'w_k.weight' in name or 'w_q.weight' in name:
                qk_decay_params.append(param)
            elif param.dim() >= 2:
                decay_params.append(param)
            else:
                nodecay_params.append(param)

        # Define optimizer groups
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0},
            {'params': qk_decay_params, 'weight_decay': qk_weight_decay}
        ]

        # Create AdamW optimizer with fused version if available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"Using fused AdamW: {use_fused}")

        return optimizer

In [16]:
# Example usage
config = GPTConfig(n_embd=768, n_head=12, n_layer=12, bias=False)                
causal_attention_block = CausalSelfAttention(config)
split_attention = SplitAttention(causal_attention_block)

# print all parameters
print(split_attention.parameters)


<bound method Module.parameters of SplitAttention(
  (w_q): Linear(in_features=768, out_features=768, bias=False)
  (w_k): Linear(in_features=768, out_features=768, bias=False)
  (w_v): Linear(in_features=768, out_features=768, bias=False)
  (c_proj): Linear(in_features=768, out_features=768, bias=False)
  (attn_dropout): Dropout(p=0.0, inplace=False)
  (resid_dropout): Dropout(p=0.0, inplace=False)
)>


In [2]:
import pytest
import torch

def test_split_attention_forward_equivalence():
    # Configuration and model setup
    config = GPTConfig(n_embd=768, n_head=12, n_layer=12, bias=True)
    causal_attention = CausalSelfAttention(config)
    split_attention = SplitAttention(causal_attention)

    # Create a sample input tensor
    batch_size, seq_length, embd_size = 3, 10, 768  # Adjust as needed
    x = torch.randn(batch_size, seq_length, embd_size)

    # Forward pass through both models
    causal_attention_output = causal_attention(x)
    split_attention_output = split_attention(x)
    
    # Assert that the outputs are the same (within a tolerance)
    assert torch.allclose(causal_attention_output, split_attention_output, atol=1e-6), "Outputs are not equivalent"

    # Assert that they have not accidentally become the same object 
    assert causal_attention is not split_attention, "causal_attention is the same object as split_attention"

# Running the test
test_split_attention_forward_equivalence()


In [3]:
import pytest
import torch
import torch.nn as nn

def test_split_attention_backward_equivalence():
    # Configuration and model setup
    config = GPTConfig(n_embd=768, n_head=12, n_layer=12, bias=True)
    causal_attention = CausalSelfAttention(config)
    split_attention = SplitAttention(causal_attention)

    # Create a sample input tensor and target
    batch_size, seq_length, embd_size = 3, 10, 768
    x_causal = torch.randn(batch_size, seq_length, embd_size, requires_grad=True)
    x_split = x_causal.clone().detach().requires_grad_(True)  # Clone and detach for separate computation

    # Define a simple loss function and a target for comparison
    target = torch.randn(batch_size, seq_length, embd_size)
    loss_fn = nn.MSELoss()

    # Forward and backward pass through causal_attention
    output_causal = causal_attention(x_causal)
    loss_causal = loss_fn(output_causal, target)
    loss_causal.backward()

    # Forward and backward pass through split_attention
    output_split = split_attention(x_split)
    loss_split = loss_fn(output_split, target)
    loss_split.backward()

    # Assert that the gradients are the same (within a tolerance)
    assert torch.allclose(x_causal.grad, x_split.grad, atol=1e-6), "Gradients are not equivalent"

    # Assert that they have not accidentally become the same object 
    assert causal_attention is not split_attention, "causal_attention is the same object as split_attention"

# Running the test
test_split_attention_backward_equivalence()


In [5]:
def test_split_gpt_wrapper_training():
    # Configuration
    config = GPTConfig(n_embd=768, n_head=12, n_layer=12, bias=True)
    weight_decay = 0.1
    qk_weight_decay = 0.0
    learning_rate = 0.001
    betas = (0.9, 0.999)
    device_type = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Create model
    split_gpt_model = SplitGPTWrapper(GPT(config))

    # Configure optimizer
    optimizer = split_gpt_model.configure_optimizers(weight_decay, qk_weight_decay, learning_rate, betas, device_type)

    # Create a sample input tensor and targets
    batch_size, seq_length = 3, 10
    x = torch.randint(0, config.vocab_size, (batch_size, seq_length))
    targets = torch.randint(0, config.vocab_size, (batch_size, seq_length))

    # Forward pass
    _, loss = split_gpt_model(x, targets)

    # Backward pass and optimizer step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Check if parameters have been updated
    for param in split_gpt_model.parameters():
        assert param.grad is not None, "Parameter gradient is None after training step"

# Run the test
test_split_gpt_wrapper_training()


number of parameters: 123.69M
Using fused AdamW: False


In [6]:
def test_optimizer_param_groups():
    # Configuration
    config = GPTConfig(n_embd=768, n_head=12, n_layer=12, bias=True)
    weight_decay = 0.1
    qk_weight_decay = 0.2
    learning_rate = 0.001
    betas = (0.9, 0.999)
    device_type = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Create models
    gpt_model = GPT(config)
    split_gpt_model = SplitGPTWrapper(gpt_model)

    # Configure optimizers
    optimizer_gpt = gpt_model.configure_optimizers(weight_decay, learning_rate, betas, device_type)
    optimizer_split_gpt = split_gpt_model.configure_optimizers(weight_decay, qk_weight_decay, learning_rate, betas, device_type)

    # Compare parameter groups
    assert len(optimizer_gpt.param_groups) == 2, "GPT does not have 2 param groups"
    assert len(optimizer_split_gpt.param_groups) == 3, "SplitGPTWrapper does not have 3 param groups"

# Run the test
test_optimizer_param_groups()


number of parameters: 123.69M
num decayed parameter tensors: 74, with 124,354,560 parameters
num non-decayed parameter tensors: 122, with 121,344 parameters
using fused AdamW: False
Using fused AdamW: False


In [7]:
def test_SplitGPTWrapper_forward_equivalence():
    # Configuration and model setup
    config = GPTConfig(n_embd=768, n_head=12, n_layer=12, bias=True)

    # Create two GPT models: one with CausalSelfAttention and one with SplitAttention
    causal_gpt_model = GPT(config)

    # Create GPT model with SplitAttention
    split_gpt_model = SplitGPTWrapper(deepcopy(causal_gpt_model))

    # Create a sample input tensor
    batch_size, seq_length = 3, 10
    x = torch.randint(0, config.vocab_size, (batch_size, seq_length))

    # Forward pass through both models
    causal_output, _ = causal_gpt_model(x)
    split_output, _ = split_gpt_model(x)

    # Assert that the outputs are the same (within a tolerance)
    assert torch.allclose(causal_output, split_output, atol=1e-6), "Outputs are not equivalent"

    # Assert that they have not accidentally become the same object 
    assert causal_gpt_model is not split_gpt_model.gpt, "causal_gpt_model is the same object as split_gpt_model"

test_SplitGPTWrapper_forward_equivalence()

number of parameters: 123.69M


In [8]:
def test_SplitGPTWrapper_backward_equivalence():
    # Configuration and model setup
    config = GPTConfig(n_embd=768, n_head=12, n_layer=12, bias=True)

    # Create two GPT models: one with CausalSelfAttention and one with SplitAttention
    causal_gpt_model = GPT(config)
    split_gpt_model = SplitGPTWrapper(deepcopy(causal_gpt_model))

    # Create a sample input tensor and targets
    batch_size, seq_length = 3, 10
    x = torch.randint(0, config.vocab_size, (batch_size, seq_length))
    targets = torch.randint(0, config.vocab_size, (batch_size, seq_length))

    # Get the word token embedding param. If the gradient is the same 
    # we can assume that the gradent is the same for all params in deeper layers as well
    param_causal = next(causal_gpt_model.parameters())
    param_split = next(split_gpt_model.gpt.parameters())

    # Assert that the initial params are equivalent
    assert torch.allclose(param_causal, param_split, atol=1e-6), "Params are not equivalent"

    # Zero gradients before backward pass
    param_causal.grad = None
    param_split.grad = None

    # Forward and backward pass through causal GPT model
    _, loss_causal = causal_gpt_model(x, targets)
    loss_causal.backward()

    # Forward and backward pass through split GPT model
    _, loss_split = split_gpt_model(x, targets)
    loss_split.backward()

    # Assert that the gradients are the same (within a tolerance)
    assert torch.allclose(param_causal.grad, param_split.grad, atol=1e-6), "Gradients are not equivalent"

    # Assert that gradient is not None and not zero
    assert param_causal.grad is not None, "Gradient is None"
    assert torch.any(param_causal.grad != 0), "Gradient is zero"
    
    # Assert that they have not accidentally become the same object 
    assert param_causal is not param_split, "param_causal is the same object as param_split"


test_SplitGPTWrapper_backward_equivalence()


number of parameters: 123.69M


In [9]:
def test_SplitGPTWrapper_optimizer_equivalence():
    
     # Configuration and model setup
    config = GPTConfig(n_embd=768, n_head=12, n_layer=12, bias=True, dropout=0.0)
    weight_decay = 0.1
    qk_weight_decay = 0.1
    learning_rate = 0.001
    betas = (0.9, 0.99)
    device_type = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Configuration and model setup
    config = GPTConfig(n_embd=768, n_head=12, n_layer=12, bias=True)

    # Create two GPT models: one with CausalSelfAttention and one with SplitAttention
    causal_gpt_model = GPT(config)
    split_gpt_model = SplitGPTWrapper(deepcopy(causal_gpt_model))

    # Configure optimizers
    optimizer_gpt = causal_gpt_model.configure_optimizers(weight_decay, learning_rate, betas, device_type)
    optimizer_split_gpt = split_gpt_model.configure_optimizers(weight_decay, qk_weight_decay, learning_rate, betas, device_type)

    # Create a sample input tensor and targets
    batch_size, seq_length = 3, 10
    x = torch.randint(0, config.vocab_size, (batch_size, seq_length))
    targets = torch.randint(0, config.vocab_size, (batch_size, seq_length))

     # Check intitial params are equivalent if we convert to SptlitWrapper
    for (name_causal, param_causal), (name_split, param_split) in zip(split_gpt_model.named_parameters(), SplitGPTWrapper(deepcopy(causal_gpt_model)).gpt.named_parameters()):
        assert torch.allclose(param_causal.data, param_split.data, atol=1e-6), f"Initial param mismatch in: {name_causal}"

    # Forward and backward pass through causal GPT model
    _, loss_causal = causal_gpt_model(x, targets)
    optimizer_gpt.zero_grad()
    loss_causal.backward()

    # Forward and backward pass through split GPT model
    _, loss_split = split_gpt_model(x, targets)
    optimizer_split_gpt.zero_grad()
    loss_split.backward()

    # Optimizer steps
    optimizer_gpt.step()
    optimizer_split_gpt.step()

    # Assert that the updated params are identical if we convert to SplitWrapper.
    # ie. Optimizer step and conversion to SplitWrapper commute
    for (name_causal, param_causal), (name_split, param_split) in zip(split_gpt_model.named_parameters(), SplitGPTWrapper(deepcopy(causal_gpt_model)).gpt.named_parameters()):
        # atol=1e-3, because it fails at 1e-4. ¯\_(ツ)_/¯.
        assert torch.allclose(param_causal.data, param_split.data, atol=1e-3), f"Updated param mismatch in: {name_causal}"

    # Assert that they have not accidentally become the same object 
    assert causal_gpt_model is not split_gpt_model.gpt, "causal_gpt_model is the same object as split_gpt_model"

test_SplitGPTWrapper_optimizer_equivalence()


number of parameters: 123.69M
num decayed parameter tensors: 50, with 124,354,560 parameters
num non-decayed parameter tensors: 98, with 121,344 parameters
using fused AdamW: False
Using fused AdamW: False


In [10]:
def test_SplitGPTWrapper_attention_layer_count():
    # Configuration and model setup
    config = GPTConfig(n_embd=768, n_head=12, n_layer=12, bias=True)

    # Create GPT model with SplitAttention
    split_gpt_model = SplitGPTWrapper(GPT(config))

    # Count SplitAttention and CausalSelfAttention layers
    split_attention_count = 0
    causal_attention_count = 0
    for block in split_gpt_model.transformer.h:
        if isinstance(block.attn, SplitAttention):
            split_attention_count += 1
        elif isinstance(block.attn, CausalSelfAttention):
            causal_attention_count += 1

    # Assert that the model contains the correct number of SplitAttention layers
    # and no CausalSelfAttention layers
    assert split_attention_count == config.n_layer, "Incorrect number of SplitAttention layers"
    assert causal_attention_count == 0, "CausalSelfAttention layers found in model"

test_SplitGPTWrapper_attention_layer_count()


number of parameters: 123.69M


In [11]:
"""
This training script can be run both on a single gpu in debug mode,
and also in a larger training run with distributed data parallel (ddp).

To run on a single GPU, example:
$ python train.py --batch_size=32 --compile=False

To run with DDP on 4 gpus on 1 node, example:
$ torchrun --standalone --nproc_per_node=4 train.py

To run with DDP on 4 gpus across 2 nodes, example:
- Run on the first (master) node with example IP 123.456.123.456:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=0 --master_addr=123.456.123.456 --master_port=1234 train.py
- Run on the worker node:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=1 --master_addr=123.456.123.456 --master_port=1234 train.py
(If your cluster does not have Infiniband interconnect prepend NCCL_IB_DISABLE=1)
"""

import os
import time
import math
import pickle
from contextlib import nullcontext

import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT

# ####################################   MODDED   ############################################

# if True, apply splitGPTwrapper!
split = True
# weight decay applied to QK matrix
qk_weight_decay = 0.0

# ##################################   END MODDED   ############################################


# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out-shakespeare-char'
eval_interval = 250
log_interval = 10
eval_iters = 200
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = False # if True, always save a checkpoint after each eval
init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
# wandb logging
wandb_log = False # disabled by default
wandb_project = 'shakespeare-char'
wandb_run_name = 'split_test' + str(time.time())
# data
dataset = 'shakespeare_char'
gradient_accumulation_steps = 1 # used to simulate larger batch sizes
batch_size = 12 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 64
# model
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
bias = False # do we use bias inside LayerNorm and Linear layers?
# adamw optimizer
learning_rate = 1e-3 # max learning rate
max_iters = 2000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.99
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 20 # how many steps to warm up for
lr_decay_iters = max_iters # should be ~= max_iters per Chinchilla
min_lr = learning_rate/10 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings
backend = 'nccl' # 'nccl', 'gloo', etc.
# system
device = 'cpu' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = False # use PyTorch 2.0 to compile the model to be faster
# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
# MODDED # exec(open('configurator.py').read()) # overrides from command line or config file
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

# various inits, derived attributes, I/O setup
ddp = int(os.environ.get('RANK', -1)) != -1 # is this a ddp run?
if ddp:
    init_process_group(backend=backend)
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc.
    seed_offset = ddp_rank # each process gets a different seed
    # world_size number of processes will be training simultaneously, so we can scale
    # down the desired gradient accumulation iterations per process proportionally
    assert gradient_accumulation_steps % ddp_world_size == 0
    gradient_accumulation_steps //= ddp_world_size
else:
    # if not ddp, we are running on a single gpu, and one process
    master_process = True
    seed_offset = 0
    ddp_world_size = 1
tokens_per_iter = gradient_accumulation_steps * ddp_world_size * batch_size * block_size
print(f"tokens per iteration will be: {tokens_per_iter:,}")

if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# poor man's data loader
data_dir = os.path.join('data', dataset)
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# attempt to derive vocab_size from the dataset
meta_path = os.path.join(data_dir, 'meta.pkl')
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout) # start with model_args from command line
if init_from == 'scratch':
    # init a new model from scratch
    print("Initializing a new model from scratch")
    # determine the vocab size we'll use for from-scratch training
    if meta_vocab_size is None:
        print("defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)")
    model_args['vocab_size'] = meta_vocab_size if meta_vocab_size is not None else 50304
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
elif init_from == 'resume':
    print(f"Resuming training from {out_dir}")
    # resume training from a checkpoint.
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    checkpoint_model_args = checkpoint['model_args']
    # force these config attributes to be equal otherwise we can't even resume training
    # the rest of the attributes (e.g. dropout) can stay as desired from command line
    for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
        model_args[k] = checkpoint_model_args[k]
    # create the model
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    # fix the keys of the state dictionary :(
    # honestly no idea how checkpoints sometimes get this prefix, have to debug more
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
    iter_num = checkpoint['iter_num']
    best_val_loss = checkpoint['best_val_loss']
elif init_from.startswith('gpt2'):
    print(f"Initializing from OpenAI GPT-2 weights: {init_from}")
    # initialize from OpenAI GPT-2 weights
    override_args = dict(dropout=dropout)
    model = GPT.from_pretrained(init_from, override_args)
    # read off the created config params, so we can store them into checkpoint correctly
    for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
        model_args[k] = getattr(model.config, k)
# crop down the model block size if desired, using model surgery
if block_size < model.config.block_size:
    model.crop_block_size(block_size)
    model_args['block_size'] = block_size # so that the checkpoint will have the right value
model.to(device)

# ####################################   MODDED   ############################################

if split:
    model = SplitGPTWrapper(model)

# ##################################   END MODDED   ############################################

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))


# ####################################   MODDED   ############################################

if split:
    optimizer = model.configure_optimizers(weight_decay, qk_weight_decay, learning_rate, (beta1, beta2), device_type)
else:
    optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)

# ##################################   END MODDED   ############################################


if init_from == 'resume':
    optimizer.load_state_dict(checkpoint['optimizer'])
checkpoint = None # free up memory

# compile the model
if compile:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model) # requires PyTorch 2.0

# wrap model into DDP container
if ddp:
    model = DDP(model, device_ids=[ddp_local_rank])

# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log and master_process:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name, config=config)

# training loop
X, Y = get_batch('train') # fetch the very first batch
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model.module if ddp else model # unwrap DDP container if needed
running_mfu = -1.0
while True:

    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0 and master_process:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if wandb_log:
            wandb.log({
                "iter": iter_num,
                "train/loss": losses['train'],
                "val/loss": losses['val'],
                "lr": lr,
                "mfu": running_mfu*100, # convert to percentage
            })
        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
    if iter_num == 0 and eval_only:
        break

    # forward backward update, with optional gradient accumulation to simulate larger batch size
    # and using the GradScaler if data type is float16
    for micro_step in range(gradient_accumulation_steps):
        if ddp:
            # in DDP training we only need to sync gradients at the last micro step.
            # the official way to do this is with model.no_sync() context manager, but
            # I really dislike that this bloats the code and forces us to repeat code
            # looking at the source of that context manager, it just toggles this variable
            model.require_backward_grad_sync = (micro_step == gradient_accumulation_steps - 1)
        with ctx:
            logits, loss = model(X, Y)
            loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation
        # immediately async prefetch next batch while model is doing the forward pass on the GPU
        X, Y = get_batch('train')
        # backward pass, with gradient scaling if training in fp16
        scaler.scale(loss).backward()
    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    scaler.update()
    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        if local_iter_num >= 5: # let the training loop settle a bit
            mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
    iter_num += 1
    local_iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break

if ddp:
    destroy_process_group()


tokens per iteration will be: 768
found vocab_size = 65 (inside data/shakespeare_char/meta.pkl)
Initializing a new model from scratch
number of parameters: 0.80M
Using fused AdamW: False


/home/wzrd/.local/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


step 0: train loss 4.1709, val loss 4.1656
iter 0: loss 4.1513, time 19497.74ms, mfu -100.00%
iter 10: loss 3.5799, time 144.70ms, mfu 0.01%
iter 20: loss 3.1266, time 149.10ms, mfu 0.01%
iter 30: loss 2.9442, time 121.35ms, mfu 0.01%
iter 40: loss 2.7845, time 154.02ms, mfu 0.01%
iter 50: loss 2.6468, time 138.55ms, mfu 0.01%
iter 60: loss 2.6535, time 179.58ms, mfu 0.01%
iter 70: loss 2.6187, time 169.52ms, mfu 0.01%
iter 80: loss 2.5915, time 167.08ms, mfu 0.01%
iter 90: loss 2.5902, time 132.19ms, mfu 0.01%


KeyboardInterrupt: 